In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas_gbq import read_gbq, to_gbq
from sklearn.preprocessing import LabelEncoder
from statsmodels.tsa.stattools import adfuller
from google.cloud import storage
from google.oauth2 import service_account

In [2]:
project_id = "wagon-bootcamp-428814"

query = """
SELECT 
    DATE(`Date`) AS Date_day,
    `Community Area`,
    COUNT(*) AS crime_count
FROM 
    `wagon-bootcamp-428814.chicago_crime.chicago_crime_tab`
GROUP BY 
    Date_day, `Community Area`
ORDER BY 
    Date_day, `Community Area`
"""

df = pd.read_gbq(query, project_id=project_id, dialect='standard')

/var/folders/ml/mfwbdmhn11qgz3hwz0b4j_2r0000gn/T/ipykernel_44522/2123151714.py:16: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df = pd.read_gbq(query, project_id=project_id, dialect='standard')


In [3]:
print("Number of missing values in 'Community Area':", df['Community Area'].isna().sum())

Number of missing values in 'Community Area': 1514


In [4]:
#sets the columns to the right data type
df['Date_day'] = pd.to_datetime(df['Date_day'])
df['Community Area'] = pd.to_numeric(df['Community Area'])

In [5]:
#creates a range of dates for the full data frame
full_date_range = pd.date_range(start=df['Date_day'].min(), end=df['Date_day'].max(), freq='D')

In [6]:
#Creates a new data frame with the missing dates
community_areas = df['Community Area'].unique()
complete_df = pd.MultiIndex.from_product([full_date_range, community_areas], names=['Date_day', 'Community Area']).to_frame(index=False)

In [7]:
#merges the old data frame with the new one
merged_df = pd.merge(complete_df, df, on=['Date_day', 'Community Area'], how='left')
merged_df['crime_count'] = merged_df['crime_count'].fillna(0)
merged_df.sort_values(by=['Date_day', 'Community Area'], inplace=True)

In [8]:
#sets the date as the index of the dt
merged_df.set_index('Date_day', inplace=True)

In [9]:
#final step to drop the lines where we dont have values if Valentina cant find the solution.
merged_df = merged_df.dropna(subset=['Community Area']) 

In [10]:
credentials = service_account.Credentials.from_service_account_file('/Users/juli/code/jonfoong/LW_chicago_crime_pred/chicago_crime/wagon-bootcamp-428814-6c8317d68431.json')

In [11]:
destination_table = 'chicago_crime.chicago_crime_test'

In [12]:
to_gbq(merged_df, destination_table, project_id=project_id, if_exists='replace', credentials=credentials)

_________________

_______________________________________________

FUNCTIONS TO CALL:

In [14]:
def call_data_frame():
    
    project_id = "wagon-bootcamp-428814"

    query = """
    SELECT 
        DATE(`Date`) AS Date_day,
        `Community Area`,
        COUNT(*) AS crime_count
    FROM 
        `wagon-bootcamp-428814.chicago_crime.chicago_crime_tab`
    GROUP BY 
        Date_day, `Community Area`
    ORDER BY 
        Date_day, `Community Area`
    """
    
    df = pd.read_gbq(query, project_id=project_id, dialect='standard')
    
    return df

In [15]:
def clean_data_frame(df):
    
    #sets the columns to the right data type
    df['Date_day'] = pd.to_datetime(df['Date_day'])
    df['Community Area'] = pd.to_numeric(df['Community Area'])

    #creates a range of dates for the full data frame
    full_date_range = pd.date_range(start=df['Date_day'].min(), end=df['Date_day'].max(), freq='D')

    #Creates a new data frame with the missing dates
    community_areas = df['Community Area'].unique()
    complete_df = pd.MultiIndex.from_product([full_date_range, community_areas], names=['Date_day', 'Community Area']).to_frame(index=False)

    #merges the old data frame with the new one
    merged_df = pd.merge(complete_df, df, on=['Date_day', 'Community Area'], how='left')
    merged_df['crime_count'] = merged_df['crime_count'].fillna(0)
    merged_df.sort_values(by=['Date_day', 'Community Area'], inplace=True)

    #sets the date as the index of the dt
    merged_df.set_index('Date_day', inplace=True)

    #final step to drop the lines where we dont have values if Valentina cant find the solution.
    merged_df = merged_df.dropna(subset=['Community Area'])

    return merged_df

In [16]:
def upload_dt_to_bigquery(merged_df, key):
    credentials = service_account.Credentials.from_service_account_file(key)
    destination_table = 'chicago_crime.chicago_crime'
    to_gbq(merged_df, destination_table, project_id="wagon-bootcamp-428814", if_exists='replace', credentials=credentials)